# Segmentez des clients d'un site e-commerce


 alimentation du Dashboard avec des requêtes qui interrogent la base de données SQL et leurs résultats 

## importations des packages et des librairies 

In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import sqlite3
import warnings

In [2]:
# Désactiver l'affichage des avertissements
warnings.filterwarnings('ignore')

In [3]:
# Configuration de la connexion à la base de données
# Connexion à la base de données
conn = sqlite3.connect("olist.db")

# Lister les tables de la base
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

# Afficher les tables
print("Tables disponibles dans la base de données :")
print(tables)

# Fermer la connexion
conn.close()


Tables disponibles dans la base de données :
            name
0      customers
1         geoloc
2    order_items
3    order_pymts
4  order_reviews
5         orders
6       products
7        sellers
8    translation


In [4]:
## la table des clients
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_customers = pd.read_sql_query("SELECT * FROM customers", conn)

# Fermer la connexion
conn.close()

In [5]:
## la table de la géolocalisation :
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_geoloc = pd.read_sql_query("SELECT * FROM geoloc", conn)

# Fermer la connexion
conn.close()

In [6]:
## la table des commandes par catégorie :
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_order_items = pd.read_sql_query("SELECT * FROM order_items", conn)

# Fermer la connexion
conn.close()

In [7]:
## la tables des payements:
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_order_pymts = pd.read_sql_query("SELECT * FROM order_pymts", conn)

# Fermer la connexion
conn.close()

In [8]:
## la table des commentaires :
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_order_reviews = pd.read_sql_query("SELECT * FROM order_reviews", conn)

# Fermer la connexion
conn.close()

In [9]:
## la table des commandes: 
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_orders = pd.read_sql_query("SELECT * FROM orders", conn)

# Fermer la connexion
conn.close()

In [10]:
## la tables des produits :
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_products = pd.read_sql_query("SELECT * FROM products", conn)

# Fermer la connexion
conn.close()

In [11]:
## la tables des vendeurs:
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_sellers = pd.read_sql_query("SELECT * FROM sellers", conn)

# Fermer la connexion
conn.close()

In [12]:
## la table de la traduction :
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger une table spécifique en DataFrame
df_translation = pd.read_sql_query("SELECT * FROM translation", conn)

# Fermer la connexion
conn.close()

## requete 1 : 
En excluant les commandes annulées, quelles sont les commandes récentes de moins de 3 mois que les clients ont reçues avec au moins 3
jours de retard ?

In [20]:
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Étape 1 : Trouver la date la plus récente de commande
max_date_query = "SELECT MAX(order_purchase_timestamp) as max_date FROM orders"
max_date = pd.read_sql_query(max_date_query, conn).iloc[0]['max_date']

# Étape 2 : Charger les commandes des 3 derniers mois
orders_recent = pd.read_sql_query(f"""
    SELECT *
    FROM orders
    WHERE order_purchase_timestamp >= DATE('{max_date}', '-3 months')
""", conn)

# Étape 3 : Filtrer les commandes valides et en retard de 3 jours ou plus
orders_filtered = orders_recent[
    (orders_recent['order_status'] != 'canceled') &
    (orders_recent['order_delivered_customer_date'].notnull()) &
    (orders_recent['order_estimated_delivery_date'].notnull())
]

# Calcul du retard
orders_filtered['delivery_delay'] = (
    pd.to_datetime(orders_filtered['order_delivered_customer_date']) -
    pd.to_datetime(orders_filtered['order_estimated_delivery_date'])
).dt.days

# Étape 4 : Garder celles avec ≥ 3 jours de retard
delayed_orders = orders_filtered[orders_filtered['delivery_delay'] >= 3]

# Affichage du résultat
delayed_orders

,index,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay
66,561,cfa4fa27b417971e86d8127cb688712f,7093250e1741ebbed41f0cc552025fd6,delivered,2018-08-16 09:44:23,2018-08-17 03:10:22,2018-08-17 18:55:00,2018-08-29 01:41:41,2018-08-22 00:00:00,7
80,711,234c056c50619f48da64f731c48242b4,44e460a655f7154ccd9faa4dbbbaf68a,delivered,2018-08-14 14:49:15,2018-08-15 03:15:15,2018-08-31 15:25:00,2018-09-01 18:14:42,2018-08-23 00:00:00,9
83,728,8ad3f1d0f96992e43566c4c82c9f6c58,948b29e24216a05fea13a18d8db45ea5,delivered,2018-07-17 21:25:29,2018-07-17 21:35:17,2018-07-18 13:08:00,2018-08-14 04:04:40,2018-08-03 00:00:00,11
190,1844,7f579e203c931f3e8410103359c6d523,d665be250d1c687c58fdea61a9b55a58,delivered,2018-08-02 18:09:27,2018-08-02 18:43:43,2018-08-10 14:15:00,2018-08-13 20:11:47,2018-08-09 00:00:00,4
204,1967,cb6e441ff2ef574ce08d3709426f88ec,4fb843d304c57182d4aa27bb39ca592b,delivered,2018-08-08 19:27:03,2018-08-08 19:44:59,2018-08-10 13:32:00,2018-08-18 01:11:58,2018-08-15 00:00:00,3
...,...,...,...,...,...,...,...,...,...,...
10110,98368,874a7690bc049bd4ce210d195bdfff7b,f6b2c83ea477af36f44a1e2df7320036,delivered,2018-08-02 12:06:47,2018-08-02 13:05:56,2018-08-10 14:42:00,2018-08-13 15:50:48,2018-08-08 00:00:00,5
10113,98382,587e32dd528769d669d539531d32aeb5,0af8a25fb0b6f833724bb5818275807b,delivered,2018-08-10 11:46:09,2018-08-11 02:50:25,2018-08-14 10:09:00,2018-09-03 09:32:31,2018-08-28 00:00:00,6
10128,98564,3adb141ba4bd69dd7fe8d3fb733c6b74,c0539d5c87fc7c97a8418adffe4b45f0,delivered,2018-08-14 23:29:21,2018-08-16 03:05:11,2018-08-16 13:28:00,2018-08-28 18:02:52,2018-08-24 00:00:00,4
10203,99279,4cf09d9e5ebbe0f91ddd7bf9aae891cd,07b6b4fe5fefb948fc76b6d2bdba77d8,delivered,2018-07-19 08:37:26,2018-07-21 03:25:17,2018-07-23 15:31:00,2018-08-21 01:12:45,2018-08-10 00:00:00,11


## Requete 2 : 
Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100000 Real sur des commandes livrées via Olist ?

In [15]:
# Connexion à la base
conn = sqlite3.connect("olist.db")

# Requête SQL
query = """
SELECT
    oi.seller_id,
    SUM(oi.price) AS total_revenue
FROM order_items oi
JOIN orders o ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY oi.seller_id
HAVING total_revenue > 100000
"""

# Exécution
high_revenue_sellers = pd.read_sql_query(query, conn)

# Résultat
high_revenue_sellers


,seller_id,total_revenue
0,1025f0e2d44d7041d6cf58b6550e0bfa,138208.56
1,1f50f920176fa81dab994f9023523100,106655.71
2,46dc3b2cc0980fb8ec44634e21d2718e,122811.38
3,4869f7a5dfa277a7dca6462dcf3b52b2,226987.93
4,4a3ca9315b744ce9f8e9374361493884,196882.12
5,53243585a1d6dc2643021fd1853d8905,217940.44
6,5dceca129747e92ff8ef7a997dc4f8ca,111126.73
7,620c87c171fb2a6dd6e8bb4dec959fc6,112461.50
8,6560211a19b47992c3666cc44a7e94c0,120702.83
9,7a67c85e85bb2ce8582c35f2203ad736,139658.69


## requete 3: 
Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) quisont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30 produits) ?

In [17]:
import pandas as pd
import sqlite3

# Connexion à la base
conn = sqlite3.connect("olist.db")

# Charger les données
df_orders = pd.read_sql_query("SELECT order_id, order_purchase_timestamp FROM orders", conn)
df_order_items = pd.read_sql_query("SELECT order_id, seller_id, shipping_limit_date FROM order_items", conn)

# Fermer la connexion
conn.close()

# Récupérer la date la plus récente d'achat
latest_date = pd.to_datetime(df_orders['order_purchase_timestamp']).max()

# Associer chaque vendeur à sa première vente
df_order_items['shipping_limit_date'] = pd.to_datetime(df_order_items['shipping_limit_date'])
first_sales = df_order_items.groupby('seller_id')['shipping_limit_date'].min().reset_index()
first_sales.rename(columns={'shipping_limit_date': 'first_sale_date'}, inplace=True)

# Calculer l'ancienneté en jours
first_sales['days_since_first_sale'] = (latest_date - first_sales['first_sale_date']).dt.days

# Garder ceux avec moins de 90 jours d'ancienneté
new_sellers = first_sales[first_sales['days_since_first_sale'] <= 90]

# Compter le nombre de produits vendus par vendeur
product_counts = df_order_items.groupby('seller_id').size().reset_index(name='products_sold')

# Fusionner les deux
active_new_sellers = pd.merge(new_sellers, product_counts, on='seller_id')

# Garder ceux qui ont vendu plus de 30 produits
engaged_new_sellers = active_new_sellers[active_new_sellers['products_sold'] > 30]

# Afficher les résultats
engaged_new_sellers


,seller_id,first_sale_date,days_since_first_sale,products_sold
43,240b9776d844d37535668549a396af32,2018-07-23 14:01:34,86,36
163,81f89e42267213cb94da7ddc301651da,2018-08-15 22:44:54,62,52
235,c9aafcd0621b2207c10e32c649cada4d,2018-07-18 21:35:19,90,52
244,d13e50eaa47b4cbe9eb81465865d8cfc,2018-08-09 09:45:18,69,69


## requete 4 :
Quels sont les 5 codes postaux, enregistrant plus de 30 reviews, avec le pire review score moyen sur les 12 derniers mois ?

In [21]:
import pandas as pd
import sqlite3

# Connexion
conn = sqlite3.connect("olist.db")

# Chargement des données
df_reviews = pd.read_sql_query("SELECT order_id, review_score, review_creation_date FROM order_reviews", conn)
df_orders = pd.read_sql_query("SELECT order_id, customer_id FROM orders", conn)
df_customers = pd.read_sql_query("SELECT customer_id, customer_zip_code_prefix FROM customers", conn)

conn.close()

# Convertir les dates
df_reviews['review_creation_date'] = pd.to_datetime(df_reviews['review_creation_date'])

# Date max pour définir la période des 12 mois
max_review_date = df_reviews['review_creation_date'].max()
start_date = max_review_date - pd.DateOffset(months=12)

# Garder les reviews sur les 12 derniers mois
recent_reviews = df_reviews[df_reviews['review_creation_date'] >= start_date]

# Joindre les reviews aux commandes
reviews_orders = pd.merge(recent_reviews, df_orders, on='order_id', how='inner')

# Puis joindre aux codes postaux clients
reviews_with_zip = pd.merge(reviews_orders, df_customers, on='customer_id', how='inner')

# Grouper par code postal
zip_scores = reviews_with_zip.groupby('customer_zip_code_prefix').agg(
    review_count=('review_score', 'count'),
    avg_score=('review_score', 'mean')
).reset_index()

# Filtrer : + de 30 reviews
zip_scores = zip_scores[zip_scores['review_count'] > 30]

# Trier par score moyen croissant (pire en haut)
worst_zips = zip_scores.sort_values(by='avg_score').head(5)

# Résultat
worst_zips


,customer_zip_code_prefix,review_count,avg_score
5652,22753,53,2.867925
5643,22723,31,3.000000
6479,28893,32,3.125000
5657,22770,38,3.184211
4138,13056,33,3.272727
